In [24]:
import sqlite3

def get_database_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema = {}
    for table in tables:
        table_name = table[0]
        # Get column names and types
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        schema[table_name] = columns

    conn.close()
    return schema


In [12]:

# Extract schema from the database
db_path = "/content/ecommerce.db"
schema = get_database_schema(db_path)

# Display schema
for table_name, columns in schema.items():
    print(f"Table: {table_name}")
    for column in columns:
        print(f"  Column: {column[1]}, Type: {column[2]}")

Table: users
  Column: user_id, Type: INTEGER
  Column: name, Type: TEXT
  Column: email, Type: TEXT
  Column: phone, Type: TEXT
  Column: address, Type: TEXT
  Column: registration_date, Type: DATE
Table: sqlite_sequence
  Column: name, Type: 
  Column: seq, Type: 
Table: products
  Column: product_id, Type: INTEGER
  Column: name, Type: TEXT
  Column: description, Type: TEXT
  Column: category, Type: TEXT
  Column: price, Type: REAL
  Column: stock, Type: INTEGER
  Column: added_date, Type: DATE
Table: orders
  Column: order_id, Type: INTEGER
  Column: user_id, Type: INTEGER
  Column: order_date, Type: DATE
  Column: total_amount, Type: REAL
  Column: status, Type: TEXT
Table: order_items
  Column: order_item_id, Type: INTEGER
  Column: order_id, Type: INTEGER
  Column: product_id, Type: INTEGER
  Column: quantity, Type: INTEGER
  Column: price_per_unit, Type: REAL


In [13]:
def format_schema(schema):
    formatted_schema = []
    for table_name, columns in schema.items():
        column_defs = [f"{col[1]} {col[2]}" for col in columns]
        formatted_schema.append(f"CREATE TABLE {table_name} ({', '.join(column_defs)});")
    return "\n".join(formatted_schema)

# Format the schema
formatted_schema = format_schema(schema)
print(formatted_schema)

CREATE TABLE users (user_id INTEGER, name TEXT, email TEXT, phone TEXT, address TEXT, registration_date DATE);
CREATE TABLE sqlite_sequence (name , seq );
CREATE TABLE products (product_id INTEGER, name TEXT, description TEXT, category TEXT, price REAL, stock INTEGER, added_date DATE);
CREATE TABLE orders (order_id INTEGER, user_id INTEGER, order_date DATE, total_amount REAL, status TEXT);
CREATE TABLE order_items (order_item_id INTEGER, order_id INTEGER, product_id INTEGER, quantity INTEGER, price_per_unit REAL);


In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model and tokenizer
model_name = "tscholak/cxmefzzi"  # T5 model fine-tuned for Text-to-SQL
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_sql(question, schema):
    input_text = f"Translate to SQL: {question} | Schema: {schema}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example questions
questions = [
    "What are the names and emails of all users?",
    "What is the average price of products?",
    "How many orders have been delivered?",
    "List all products in the Electronics category."
]

# Convert questions to SQL
for question in questions:
    sql_query = generate_sql(question, formatted_schema)
    print(f"Question: {question}")
    print(f"SQL Query: {sql_query}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

Question: What are the names and emails of all users?
SQL Query: Translate to SQL: select name, email from users

Question: What is the average price of products?
SQL Query: translate to SQL: select avg(price) from products

Question: How many orders have been delivered?
SQL Query: CREATE TABLE orders ( count(*) from orders where order_date_dATE > "DATE"

Question: List all products in the Electronics category.
SQL Query: translate to sqlite3: select name from products where category = 'Electronics'



In [25]:
def generate_sql(question, schema, tokenizer, model):
    # Format the input prompt
    input_text = f"Translate to SQL: {question} | Schema: {schema}"

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the output
    outputs = model.generate(**inputs)

    # Decode the output and remove the prompt (if any)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove unwanted prefixes
    prefixes = ["Translate to SQL:", "translate to SQL:", "translate to sqlite3:"]
    for prefix in prefixes:
        if sql_query.startswith(prefix):
            sql_query = sql_query[len(prefix):].strip()

    return sql_query

In [20]:
# Example questions
questions = [
    "What are the names and emails of all users?",
    "What is the average price of products?",
    "How many orders have been delivered?",
    "List all products in the Electronics category."
]

# Convert questions to SQL
for question in questions:
    # Pass tokenizer and model to generate_sql
    sql_query = generate_sql(question, formatted_schema, tokenizer, model)
    print(f"Question: {question}")
    print(f"SQL Query: {sql_query}\n")

Question: What are the names and emails of all users?
SQL Query: select name, email from users

Question: What is the average price of products?
SQL Query: select avg(price) from products

Question: How many orders have been delivered?
SQL Query: CREATE TABLE orders ( count(*) from orders where order_date_dATE > "DATE"

Question: List all products in the Electronics category.
SQL Query: select name from products where category = 'Electronics'



In [26]:
def execute_sql_query(sql_query, db_path):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Validate the query (ensure it's a SELECT query)
        if not sql_query.strip().lower().startswith("select"):
            raise ValueError("Only SELECT queries are allowed for safety reasons.")

        # Execute the query
        cursor.execute(sql_query)

        # Fetch results and column names
        rows = cursor.fetchall()
        columns = [description[0] for description in cursor.description]  # Get column names

        # Close the connection
        conn.close()

        return rows, columns

    except sqlite3.Error as e:
        # Handle database errors
        print(f"Database error: {e}")
        return None, None
    except ValueError as e:
        # Handle invalid queries
        print(f"Invalid query: {e}")
        return None, None
    except Exception as e:
        # Handle other errors
        print(f"Unexpected error: {e}")
        return None, None


In [21]:
# Execute the queries
for question in questions:
    print(f"Question: {question}")

    # Convert question to SQL
    sql_query = generate_sql(question, formatted_schema, tokenizer, model)
    print(f"Generated SQL Query: {sql_query}")

    # Execute the query
    try:
        result, columns = execute_sql_query(sql_query, db_path)

        if result is not None and columns is not None:
            print("Query Results:")
            print("-" * 50)
            print(" | ".join(columns))  # Print column headers
            print("-" * 50)
            for row in result:
                print(" | ".join(map(str, row)))  # Print each row
            print("-" * 50)
        else:
            print("No results or an error occurred while executing the query.")

    except Exception as e:
        print(f"Error executing query: {e}")

    print("\n" + "=" * 50 + "\n")

Question: What are the names and emails of all users?
Generated SQL Query: select name, email from users
Query Results:
--------------------------------------------------
name | email
--------------------------------------------------
Eunice Muriithi | eunicemuriithi034@gmail.com
Peter Kamau | mwangikamau@gmail.com
Christine Atieno | akinyiatieno0@gmail.com
Omollo Ochieng | omolloochieng09@gmail.com
Jane Njeri | wanjikunjeri@gmail.com
James Mwangi | jamesmwangi@gmail.com
Linda Wambui | lindawambui@gmail.com
Samuel Otieno | samuelotieno@gmail.com
Grace Njiru | gracenjiru@gmail.com
Mark Otieno | markotieno@gmail.com
--------------------------------------------------


Question: What is the average price of products?
Generated SQL Query: select avg(price) from products
Query Results:
--------------------------------------------------
avg(price)
--------------------------------------------------
49000.0
--------------------------------------------------


Question: How many orders have bee